# Exercise 2 - Pre-trained models

### Applying already pre-trained models

We have previously trained a bodypart predictor, that can classify one of the 19 following bodyparts:

In [ ]:
class_map = {0: 'abdomen',
             1: 'ankle',
             2: 'buttocks',
             3: 'elbow',
             4: 'face',
             5: 'foot',
             6: 'groin',
             7: 'hand',
             8: 'knee',
             9: 'lower_arm',
             10: 'lower_back',
             11: 'lower_leg',
             12: 'neck',
             13: 'shoulder',
             14: 'thorax',
             15: 'upper_arm',
             16: 'upper_back',
             17: 'upper_leg',
             18: 'wrist'}

### Loading the model
Keras provies a very good model format that is easily saveable and loadable.

We can load the model and inspect as we've done previously:

In [ ]:
from keras.models import load_model

model = load_model('bodypart_predictor.h5')

## This is a big model, have a look by un-commenting this line
model.summary()


We can see the shape of the output of a model by calling:

In [ ]:
model.outputs

This tells us that the model will give us a vector of shape `(?, 19)`. The question mark will take on the number of images that we pass through on a given pass of the model. So if we evaluate 3 images we'll get a 3 x 19 matrix out.

The 19 is the number of bodyparts, and the value in this index of the vector will describe the probability that the image contains a given bodypart.

### Let's try applying it to an image

Now we need to apply the model to an image. This requires that we do a little bit of pre-processing of the image.
We can load an image from the disk:

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img

image = load_img('bodyparts_data/hand/fingers.jpg', target_size=(224, 224))
image

In [ ]:
# convert it to array
img = img_to_array(image)
img

The model is transfer learned from a model trained on ImageNet where they normalize the images by subtracting the means of each of the color channels (RGB), so to use the model we need to do the same:

In [ ]:
import numpy as np

def imagenet_norm(image):
    # Mean RGB values. Do not change these values!
    vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1, 1, 3))

    # Normalize image
    image = image - vgg_mean
    image = image[:, :, ::-1]  # Reverse axis rgb->bgr

    return image

img = imagenet_norm(img)
array_to_img(img)

Looks a bit wierd, but this is what the model is used to - let's try and run it!

In [ ]:
# The image is shape 224x224x3 but the model expects a tensor of Nx224x224x3,
# where N is the number of images it has to predict on. So to predict we must
# wrap the image in an array:
prediction = model.predict(np.array([img]))
prediction

This output is the probability for each of the classes.

Lets' find the highest one:

In [ ]:
class_index = np.argmax(prediction)
class_index 

This is the index from the `class_map` defined in the top:

In [ ]:
class_map[class_index]

YAY! that was the class we were looking for!

### Exercise A:
Now it's your turn, try taking some photos with your phone and upload them by going back to the folder tab and clicking "upload".
Try changing the path to be your image instead. Below we defined a small helper function to apply a model to an image.

Please sort your data according to the classes in `bodyparts_data/`.

* Question 1: Can you take a picture where the model is not classifying correctly?

* Question 2: What happens when you have multiple bodyparts in an image?

* Question 3: What happens when there are no bodyparts in the image?

* Question 4: Can you change the function below so it prints the certainty for each of the classes, not just the predicted one?


In [ ]:
def apply_model(image_path):
    # Load and normalize
    img = load_img(image_path, target_size=(224, 224))
    norm_img = imagenet_norm(img_to_array(img))
    
    # Apply model and get class results
    class_pred = model.predict(np.array([norm_img]))
    proba = np.max(class_pred)
    class_name = class_map[np.argmax(class_pred)]
    
    # Print output and return image to be displayed
    print('The predicted class for this image is %s, with %0.2f certainty:' % (class_name, proba))
    return img

apply_model('bodyparts_data/hand/fingers.jpg')

In [ ]:
apply_model("<< Insert the path to your own image here >>")

## Training a model

You might have found that some of the images didn't match the prediction that you wanted, so if you want you can try and get enough to actually do some training of your own - starting from the model we provided.

To do this you need to sort the images you've taken into folders with the names of the class, like so:

In [ ]:
!tree bodyparts_data/

Put the images you've taken of each class into the corresponding folder under `data/`

Now you need to define a generator as you did in exercise 1:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(preprocessing_function=imagenet_norm)

# Create an iterator that can loop through the data
directory_iterator = image_data_generator.flow_from_directory('bodyparts_data/',
                                                              target_size=(224, 224))

### Exercise B:

Now to train a model on the data you've collected you just need to run `model.fit_generator`. Try as before playing around with a different  number of epochs. You're not training on a lot of data. 

* Question 1: Can you get the model to be better at your images.

* Question 2: What happens when you apply your newly trained model to new images you take that hasn't been seen before?

In [ ]:
model.fit_generator(directory_iterator,
                    epochs=3
                   )